# Analyzing your Datasets
---


## Table of Contents

- [Introduction](#Introduction)
    - [Learning Objectives](#Learning-Objectives)
    - [Methods](#Methods)
- [Python Setup](#Python-Setup)
- [Load the Data](#Load-the-Data)
    - [Establish a Connection to the Database](#Establish-a-Connection-to-the-Database)
    - [Pull Data from the Database](#Pull-Data-from-the-Database)
    - [Rename the Variables](#Rename-the-Variables)
- [Analysis](#Analysis)
- [Getting to know the IDHS database](#Getting-to-know-the-IDHS-database)

## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In an ideal world, we will have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). 
However, that is hardly ever true - and we have to work with using our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF, and we will use our datasets to answer some questions of interest. 

### Learning Objectives
This notebook will give you the opportunity to spend some hands-on time with the data. 

You will have an opportunity to explore the different datasets in the ADRF, and this notebook will take you around the different ways you can analyze your data. 
This involves looking at basic metrics in the larger dataset, taking a random sample, creating derived variables, making sense of the missing values & so on. 

We will be mostly using SQL (via psycopg2), hence giving you the opportunity to interact with the database directly. The same queries can also be handled by Pandas in Python (by converting your datasets into dataframes). 

After going through this notebook, you will have a good understanding around: 

- How to create new tables of interest from the larger tables in database
- How to decide on the variables of interest
- How to quickly look through aggregate metrics before proceeding with analysis
- Possible pitfalls
- How to handle missing values
- How to join newly created tables
- How to think about caveats in your final results

### Methods
We will be using the `psycopg2` Python package to access tables in our class database  server - PostgreSQL. 

To read the results of our queries, we will be using the `pandas` Python package, which has the ability to read tabular data from SQL queries into a pandas DataFrame object. Within `pandas`, we will use various commands:

- Subsetting data
- `groupby`
- `merge`

Within SQL, we will use various queries:

- `CREATE TABLE`
- `SELECT ROWS`
- Summing over groups
- Counting distinct values of desired variables
- Ordering data by chosen variables
- Selecting a random sub-sample

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

In Python, we `import` packages. The `import` command allows us to use libraries created by others in our own work by "importing" them. You can think of importing a library as opening up a toolbox and pulling out a specific tool. 
- NumPy is short for numerical python. NumPy is a lynchpin in Python's scientific computing stack. Its strengths include a powerful *N*-dimensional array object, and a large suite of functions for doing numerical computing. 
- Pandas is a library in Python for data analysis that uses the DataFrame object from R which is similiar to a spreedsheet but allows you to do your analysis programaticaly rather than the point-and-click of Excel. It is a lynchpin of the PyData stack.  
- Psycopg2 is a python library for interfacing with a PostGreSQL database. 
- Matplotlib is the standard plotting library in python. 
`%matplotlib inline` is a so-called "magic" function of Jupyter that enables plots to be displayed inline with the code and text of a notebook. 

In [2]:
# general use imports
import datetime
import glob
import inspect
import numpy
import os
import six
import warnings

# pandas-related imports
import pandas as pd
import sqlalchemy

# CSV file reading-related imports
import csv

# database interaction imports
import psycopg2
import psycopg2.extras

__When in doubt, use shift + tab to read the documentation of a method.__

__The `help()` function provides information on what you can do with a function.__

## Load the Data
- Back to [Table of Contents](#Table-of-Contents)

Instead of using pgAdmin or the command line sql too directly, we can also carry out sql queries using python. But more power of python and pandas comes from that they can greatly facilitate descpritive statistics of the data, which is rather complicated to do, if not possible, in sql per se. Moreover, python and pandas plus matplotlib package can create data visualizations that greatly helps data analysis. We will see some of these advantages in the following content.

Pandas provides many ways to load data. It allows the user to read the data from a local csv or excel file, or pull the data from a relational database. Since we are working with the relational database appliedda in this course, we will demonstrate how to use pandas to read data from a relational database. For examples to read data from a csv file, refert to the pandas documentation [Getting Data In/Out](pandas.pydata.org/pandas-docs/stable/10min.html#getting-data-in-out).

The function to create a sql query and put the data into a pandas dataframe (more to come) is `pd.read_sql()`. Just like doing a sql query from pgAdmin, this function will ask for some information about the database, and what query you woul like to run. Let's walk through the example below.

### Establish a Connection to the Database with `psycopg2`
In the most simple case, only 2 parameters are required by the `pd.read_sql()` function to pull data. The first parameter is the connection to the database. To create a connection we need to use the psycopg2 package and tell it which database and which host we want to connect to, just like in pgAdmin.

#### Parameter 1: Connection

In [2]:
# to create a connection to the database, we need to pass the name of the database and host of the database
# db_name = "appliedda"
# db_host = "10.10.2.10"
# conn = psycopg2.connect(database=db_name, host=db_host) #database connection

#### Parameter 2: Query
This part is similar to writing a sql query in pgAdmin. Depending on what data we are interested in, we can use different queries to pull different data. In this example, we will pull all the content of wage_person data.

In [4]:
# query = '''
# SELECT *
# FROM idhs.hh_indcase_spells h
# WHERE h.start_date >= '2015-01-01' AND h.end_date <= '2015-03-31'
# '''

Note:

- the three quotation marks surrounding the query body is called multi-line string. It is quite handy for writing sql queries because the new line character will be considered part of the string, instead of breaking the string

### Pull Data from the Database
Now that we know what the arguments are for the query, we can pass them to the `pd.read_sql()` function, and obtain the data.

In [7]:
# here we pass the query and the connection to the pd.read_sql() function and assign the variable `business_licenses` 
# to the dataframe returned by the function
# business_licenses = pd.read_sql(query, conn)

### Rename the Variables

In [14]:
# business_licenses.columns = ['business_activity', 'address', 'legal_name', 'dba_name', 'filing_period']

## <span style="background-color: #FFFF00"> TEMP </span>

In [2]:
business_licenses = pd.read_csv('../../data/KCMO/BusinessLicense2013_2018NYU_01222018.csv')
# od_main_JT01 = pd.read_csv('../../data/LODES/mo_od_main_JT01.csv')
xwalk = pd.read_csv('../../data/LODES/raw/mo_xwalk.csv')

In [5]:
business_licenses = business_licenses[business_licenses['fdtmfilingPeriod']=='12/31/17'].reset_index(drop = True)

In [1]:
wac = wac[wac['cbsaname']=="Kansas City, MO-KS"].reset_index(drop = True)

In [ ]:
business_licenses.columns = ['business_activity', 'address', 'legal_name', 'dba_name', 'filing_period']

## Analysis
- Back to [Table of Contents](#Table-of-Contents)

__What is the current distribution of jobs by industrial sector across Kansas City, MO? What is the trend over the last 10 years?__

__Other interesting questions we can answer using same/similar datasets__
- How many blocks have industry jobs in Kansas City, MO?
- To what extent to the different counties that make up Kansas City, MO, differ in job?
- Distribution of these jobs by gender, race, age, income.

### Step 1: What is in the database?

In this preliminary step, you will have a chance to discover the datasets in the ADRF that we presented this morning. These include the Census LODES data, KCMO water services data, and more.

__ LODES Data: Workplace Area Characteristics File__

In [4]:
# query = '''
# SELECT *
# FROM mo_wac
# LIMIT 100
# '''

# wac = pd.read_sql(query, conn)

In [6]:
# TEMP
wac = pd.read_csv('../../data/LODES/mo_wac_S000_JT01.csv')
wac['w_geocode'] = wac['w_geocode'].astype(str)

In [65]:
wac.head()

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate,year
0,290019501001008,3,0,3,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,20160219,2014
1,290019501001022,2,0,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,20160219,2014
2,290019501001025,2,0,2,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,20160219,2014
3,290019501001055,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,20160219,2014
4,290019501001128,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,20160219,2014


In [9]:
wac.columns

Index(['w_geocode', 'C000', 'CA01', 'CA02', 'CA03', 'CE01', 'CE02', 'CE03',
       'CNS01', 'CNS02', 'CNS03', 'CNS04', 'CNS05', 'CNS06', 'CNS07', 'CNS08',
       'CNS09', 'CNS10', 'CNS11', 'CNS12', 'CNS13', 'CNS14', 'CNS15', 'CNS16',
       'CNS17', 'CNS18', 'CNS19', 'CNS20', 'CR01', 'CR02', 'CR03', 'CR04',
       'CR05', 'CR07', 'CT01', 'CT02', 'CD01', 'CD02', 'CD03', 'CD04', 'CS01',
       'CS02', 'CFA01', 'CFA02', 'CFA03', 'CFA04', 'CFA05', 'CFS01', 'CFS02',
       'CFS03', 'CFS04', 'CFS05', 'createdate', 'year'],
      dtype='object')

Take some time to look at the documentation and understant what the different column names refer to.

__ LODES Data: Crosswalk File __

In [ ]:
# query = '''
# SELECT *
# FROM mo_xwalk
# '''

# xwalk = pd.read_sql(query, conn)

In [17]:
# TEMP
xwalk = pd.read_csv('../../data/LODES/mo_xwalk.csv')
xwalk = xwalk.astype(str)

/Users/nj995/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
xwalk.describe(include = 'all')

,tabblk2010,st,stusps,stname,cty,ctyname,trct,trctname,bgrp,bgrpname,...,stanrcname,necta,nectaname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate
count,343565,343565,343565,343565,343565,343565,343565,343565,343565,343565,...,343565,343565,343565,343565,343565,343565,343565,343565,343565,343565
unique,343565,1,1,1,115,115,1393,1393,4506,4506,...,1,1,1,10,10,14,14,341645,342168,1
top,291059602985179,29,MO,Missouri,29189,"St. Louis County, MO",29007950200,"9502 (Audrain, MO)",290414701002,"2 (Tract 4701, Chariton, MO)",...,nan,99999,nan,9999999999999999999999,nan,29290016,5/9 Central Region WIB,37.2056999,-94.5382808,20170919
freq,1,343565,343565,343565,18737,18737,1401,1401,755,755,...,343565,343565,343565,342094,342094,49276,49276,4,3,343565


Again, take some time to look at the documentation and understand all the levels of geography in the crosswalk file.

__Water Services: Consumption Data__

In [ ]:
# query = '''
# SELECT *
# FROM water
# LIMIT 100
# '''

# water = pd.read_sql(query, conn)

### Step 2: Summary Statistics in Different Datasets

In this section, let's start looking at aggregate statistics on the data. We are interested in the distribution of jobs by industrial classification, so let's take a look at the overall distribution in 2015.

In [20]:
# query = '''
# SELECT
#     year
#     , sum(CNS01) as CNS01
#     , sum(CNS02) as CNS02
#     , sum(CNS03) as CNS03
#     , sum(CNS04) as CNS04
#     , sum(CNS05) as CNS05
#     , sum(CNS06) as CNS06
#     , sum(CNS07) as CNS07
#     , sum(CNS08) as CNS08
#     , sum(CNS09) as CNS09
#     , sum(CNS10) as CNS10
#     , sum(CNS11) as CNS11
#     , sum(CNS12) as CNS12
#     , sum(CNS13) as CNS13
#     , sum(CNS14) as CNS14
#     , sum(CNS15) as CNS15
#     , sum(CNS16) as CNS16
#     , sum(CNS17) as CNS17
#     , sum(CNS18) as CNS18
#     , sum(CNS19) as CNS19
#     , sum(CNS20) as CNS20
# FROM mo_wac
# GROUP BY year
# LIMIT 100
# '''
# wac_year_stats = pd.read_sql(query, conn)

In [28]:
# TEMP:
filter_col = [col for col in wac if col.startswith('CN')]
wac_year_stats = wac_year_stats.groupby('year')[filter_col].sum()

In [29]:
wac_year_stats

,CNS01,CNS02,CNS03,CNS04,CNS05,CNS06,CNS07,CNS08,CNS09,CNS10,CNS11,CNS12,CNS13,CNS14,CNS15,CNS16,CNS17,CNS18,CNS19,CNS20
year,,,,,,,,,,,,,,,,,,,,
2002,10397.0,3827.0,13979.0,136760.0,311954.0,112754.0,279573.0,88867.0,59828.0,100939.0,37023.0,109010.0,48706.0,112376.0,220609.0,294074.0,37138.0,187232.0,72553.0,110106.0
2003,10034.0,3716.0,16832.0,139420.0,293223.0,113426.0,282092.0,92457.0,58018.0,103069.0,36392.0,107358.0,59556.0,118067.0,206306.0,295460.0,45264.0,190848.0,76228.0,93128.0
2004,9817.0,4097.0,17762.0,143734.0,290710.0,111971.0,281492.0,88402.0,57023.0,103500.0,36279.0,108889.0,63204.0,121280.0,205007.0,302838.0,45198.0,195623.0,75942.0,87468.0
2005,9900.0,4616.0,17754.0,144243.0,288224.0,115217.0,288305.0,89998.0,57488.0,104219.0,36657.0,111530.0,67252.0,125585.0,207191.0,307181.0,46072.0,200849.0,77264.0,88020.0
2006,10856.0,4779.0,18016.0,150242.0,285360.0,115831.0,287137.0,93054.0,55294.0,111344.0,35703.0,116682.0,67378.0,130874.0,209107.0,308742.0,46327.0,203765.0,76123.0,87835.0
2007,11089.0,4892.0,18658.0,149831.0,286289.0,118608.0,287227.0,92386.0,55217.0,114034.0,36476.0,120387.0,67940.0,129533.0,219910.0,327249.0,45861.0,206748.0,75979.0,90666.0
2008,10255.0,4817.0,19453.0,150336.0,281021.0,121869.0,281293.0,93378.0,52701.0,111795.0,37023.0,126551.0,65895.0,132946.0,228224.0,339845.0,46058.0,211911.0,78319.0,93324.0
2009,10183.0,4163.0,20002.0,128946.0,246021.0,115904.0,271990.0,87496.0,51565.0,109001.0,35085.0,120245.0,62247.0,119204.0,234191.0,356447.0,44981.0,204969.0,76893.0,95289.0
2010,10367.0,4214.0,19600.0,114038.0,231363.0,116023.0,272633.0,83644.0,49810.0,110198.0,33794.0,119009.0,63438.0,129147.0,225718.0,367974.0,44738.0,207802.0,76995.0,118824.0


We can change there values into the percentage of all jobs that year:

In [32]:
wac_year_stats['total_jobs'] = wac_year_stats.sum(axis=1)
for var in filter_col:
    wac_year_stats[var] = wac_year_stats[var]/wac_year_stats['total_jobs']

In [33]:
wac_year_stats

,CNS01,CNS02,CNS03,CNS04,CNS05,CNS06,CNS07,CNS08,CNS09,CNS10,...,CNS12,CNS13,CNS14,CNS15,CNS16,CNS17,CNS18,CNS19,CNS20,total_jobs
year,,,,,,,,,,,,,,,,,,,,,
2002,0.002214,0.000815,0.002977,0.029126,0.066438,0.024014,0.059542,0.018926,0.012742,0.021497,...,0.023216,0.010373,0.023933,0.046984,0.062630,0.007909,0.039876,0.015452,0.023450,4695410.0
2003,0.002143,0.000794,0.003595,0.029779,0.062631,0.024227,0.060253,0.019748,0.012392,0.022015,...,0.022931,0.012721,0.025218,0.044066,0.063108,0.009668,0.040764,0.016282,0.019892,4681788.0
2004,0.002089,0.000872,0.003779,0.030579,0.061847,0.023821,0.059886,0.018807,0.012131,0.022019,...,0.023166,0.013446,0.025802,0.043614,0.064427,0.009616,0.041618,0.016156,0.018608,4700472.0
2005,0.002073,0.000967,0.003718,0.030207,0.060359,0.024129,0.060376,0.018847,0.012039,0.021825,...,0.023356,0.014084,0.026300,0.043390,0.064329,0.009648,0.042061,0.016181,0.018433,4775130.0
2006,0.002248,0.000990,0.003731,0.031113,0.059094,0.023987,0.059462,0.019270,0.011451,0.023058,...,0.024163,0.013953,0.027102,0.043303,0.063936,0.009594,0.042197,0.015764,0.018189,4828898.0
2007,0.002255,0.000995,0.003794,0.030466,0.058213,0.024117,0.058404,0.018785,0.011228,0.023187,...,0.024479,0.013815,0.026339,0.044716,0.066542,0.009325,0.042039,0.015449,0.018436,4917960.0
2008,0.002062,0.000968,0.003911,0.030224,0.056498,0.024501,0.056552,0.018773,0.010595,0.022476,...,0.025442,0.013248,0.026728,0.045883,0.068324,0.009260,0.042603,0.015746,0.018762,4974028.0
2009,0.002126,0.000869,0.004176,0.026922,0.051365,0.024199,0.056787,0.018268,0.010766,0.022758,...,0.025105,0.012996,0.024888,0.048895,0.074420,0.009391,0.042794,0.016054,0.019895,4789644.0
2010,0.002160,0.000878,0.004084,0.023765,0.048214,0.024178,0.056814,0.017431,0.010380,0.022964,...,0.024800,0.013220,0.026913,0.047038,0.076683,0.009323,0.043304,0.016045,0.024762,4798658.0


We are also specially interested in the specific region of Kansas City, MO. Let's take a look at the crosswalk file and identify what geography is most relevant. 

After looking at the documentation, the unique Place Code and Place Name seems to best delimit Kansas City, MO.

In [45]:
xwalk_kcmo = xwalk[xwalk['stplcname']=="Kansas City city, MO"]

In [50]:
xwalk_kcmo.describe(include = 'all')

,tabblk2010,st,stusps,stname,cty,ctyname,trct,trctname,bgrp,bgrpname,...,stanrcname,necta,nectaname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate
count,12001,12001,12001,12001,12001,12001,12001,12001,12001,12001,...,12001,12001,12001,12001,12001,12001,12001,12001,12001,12001
unique,12001,1,1,1,4,4,172,172,459,459,...,1,1,1,3,3,1,1,11981,11966,1
top,290950155002083,29,MO,Missouri,29095,"Jackson County, MO",29095015500,"155 (Jackson, MO)",290950155002,"2 (Tract 155, Jackson, MO)",...,nan,99999,nan,9999999999999999999999,nan,29290003,003 Kansas City Vicinity Region WIB,39.0040916,-94.5382808,20170919
freq,1,12001,12001,12001,8583,8583,303,303,248,248,...,12001,12001,12001,11985,11985,12001,12001,2,3,12001


We notice above that Kansas City, MO, is made up of:

- 12001 blocks (`tabblk2010`)
- 4 counties (`cty`)
- 62 zip codes (`zcta`)

<span style="background-color: #FFFF00"> add something similar for water </span>

### Step 3: Combine datasets

While the LODES data gives interesting information about the distribution of jobs by industry at block level over the entire Missouri state, we would like to restrict our analysis to the city of Kansas City. Unfortunately there is no metropolitan area information on the LODES dataset. The only way of restricting to Kansas City is to first merge on the geographic information from the crosswalk file.

The following SQL query will directly merge on the relevant geographic information, and restrict to the value of interest (where `stplcname` is "Kansas City city, MO").

In [53]:
# query = '''
# SELECT
#     a.*
#     , b.tabblk2010
#     , b.cty
#     , b.ctyname
#     , b.stplc
#     , b.stplcname
# FROM mo_wac AS a
# LEFT JOIN mo_xwalk AS b
# ON a.w_geocode = b.tabblk2010
# WHERE stplc = "Kansas City city, MO";
# '''

# wac_kcmo = pd.read_sql(query, conn)

In [70]:
# TEMP:
wac_kcmo = pd.merge(wac, xwalk[['tabblk2010', 'cty', 'ctyname', 'stplc', 'stplcname']], how = 'left'
                    , left_on = 'w_geocode', right_on = 'tabblk2010')
wac_kcmo = wac_kcmo[wac_kcmo['stplcname'] == "Kansas City city, MO"].reset_index(drop = True)

Now we can conduct the same analysis as before on the the area of Kansas City, MO.

<span style="background-color: #FFFF00"> TO DO </span>

### Step 4: Comparing Variables from different Datasets

<span style="background-color: #FFFF00"> TO DO with water data or wage records </span>

## Steps

### Step 1: Looking for datasets, variables of interest

In the LODES data:
- Overall Jobs
    - See variable `C000` for total jobs by Census Block.
- Worker Classification:
    - Total job count is split by worker age (`CA01` to `CA03`), worker wage (`CE01` to `CE03`), worker race (`CR01` to `CR05`), worker ethnicity (`CT01` to `CT02`), worker education (`CD01` to `CD04`), and worker sex (`CS01` to `CS02`).
- Industry Classification:
    - The number of jobs by NAICS (North American Industry Classification System) Codes are given in variables `CNS01` to `CNS20`.
    - *Some NAICS Codes are grouped together when they refer to a same sector (codes 31 to 33 all are all manufacturing codes, for example)*
    - Look in the ADRF for the <span style="background-color: #FFFF00">*2007 North American Industry Classification System (NAICS) Definitions data file* </span>. We have the mapping of variables `CNS01` to `CNS20` to the NAICS classification code.
- Year Variable
    - See `year` variable.
- Geography
    - The `w_geocode` is the workplace Census Block Code. 
    
In order to map the Census Block Code to larger geographic areas, a crosswalk table is also provided (`mo_xwalk`). The has been mapped to:
- state level (`st`, `stusps`, `stname`)
- county level (`cty`, `ctyname`)
- metropolitan area level (`cbsa`, `cbsaname`)
- zipcode (`zcta`)

       
### Step 2: Getting/Creating relevant data tables
- [DATA A](#DATA-A): Identify how many blocks in the Kansas City, MO, metropolitan area had jobs in 2017. In which block were there the highest number of jobs?
- [DATA B](#DATA-B): What NAICS Industry Code employed the most people in 2017?


- [DATA C](#DATA-C): Getting a random sample of 10,000 rows from wage data
- [DATA D](#DATA-D): Matching DATA A and DATA C
- [DATA E](#DATA-E): MATCHING Data D HH_Member data to subset information only for head of households
- [DATA F](#DATA-F): Comparing avergae wages across sex

### DATA A
- back to [Steps](#Step-1:-Looking-for-the-variables-of-interest-in-the-LODES-data)

Getting a list of all employers in manufacturing industries in 2010. We will use the dataset "ides.il_qcew_employers" for this

In [ ]:
# generate SQL
sql_string = "CREATE table if not exists " + output_schema + "." + table_unique_prefix + "naics_2010 as"
sql_string += " SELECT DISTINCT empr_no, seinunit, ein, name_legal, auxiliary_naics "
#chosen the variables of interest
sql_string += " from ides.il_qcew_employers "
sql_string += " where substr(auxiliary_naics,1,2) in ('31', '32', '33')"
#subset the data to contain only those values whose naics value starts from 31, 32 or 33
sql_string += " and year=2010 and quarter=1"
#subset the data for the time period of interest
sql_string += ";"
#this seim-colon is not necessary in psycopg2 but it is widely used across SQL supporting programs. 

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE NAICS_2010 created on " + str( datetime.datetime.now() ) )

#Rollback query below if you need it
#pgsql_connection.rollback()

In [ ]:
#Now that we created our table, let us look at some of the top rows
#generating read SQL
sql_string1 = "select * from " + output_schema +  "." + table_unique_prefix + "naics_2010 limit 10"
#since we gave a limit 10 option in our query, it is not necessary to use the head command below
#(unless you want to see fewer than 10 rows)

# read it
r = pd.read_sql(sql_string1, con=pgsql_connection)
r.head(10)

In [ ]:
#It is likely that you will see that some employers do not have a legal name. 
#While not necessary for this aggregate analysis- this might be a question of interest before proceeding. 


#Let us find how many employers do not have a legal name?

#generating read SQL
sql_string2 = "select count(distinct empr_no) from " + output_schema + "." +  table_unique_prefix + "naics_2010 " 
sql_string2+= " WHERE name_legal in ('nan', '.', '') "
# read it
r = pd.read_sql(sql_string2, con=pgsql_connection)
r



In [ ]:
#It is important to see how many total number of employers are there in the data. 
#This is to take the value of missing legal names into 'statistical perspective'

#generating read SQL
sql_string3 = "select count(distinct empr_no) from " + output_schema + "." + table_unique_prefix + "naics_2010 "

# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


#### Missing Values

- XXX of our data has missing legal names
- A good rule of thumb is that we can drop data with missing values if missing values is less than 5% of the data. 
    - However, in this case: 
     - a) our variable of interest 'name_legal' is not important to us
     - b) and missing values are very high

**So we will not drop the data**

*An alternative and probably better way to see if the employers with no legal names can be dropped is to see the below: 
    1. what percentage of employees work in these firms
    2. how many wages are earned by the employees working with these employers
This will again be only necessary if we care about this variable to begin with. 
Since this is an aggregate level study- we do not care about individual employers or their names. 
However, it could have been a variable of interest in some other study*

- We could have done the same analysis for other variables of interest also such as: 
    - seinunit or ein
    - naics (but it will not be missing since we subsetted our data for manufacturing codes initially)

### USING A CONDITION STATEMENT IN SQL

##### Of interest: If an employer has a legal name or not
#### SQL QUERIES USED: 
    * CASE WHEN
    * GROUP BY
    * SUM
    * OVER
    * DISTINCT


In [ ]:
##WE COULD HAVE DONE THE SAME THING IN ONE QUERY ALSO
#Calculating the percentage of employers having a legal name vs the ones not

#CREATING AN ID with a CONDITION STATEMENT (CALLED CASE in SQL)
sql_string3 = "select case when name_legal in ('nan', '.', '') then 1 else 2 end as name_type, "
#Creating an ID with a value of 1 if name_legal is missing and a value of 2 if not missing
sql_string3 += "count(distinct empr_no), "
#Counting the distinct number of employers associated with both IDs
sql_string3 += "count(distinct empr_no)/(sum(count(distinct empr_no)) over()) PER "
#Counting the percentage of values in both IDs
sql_string3 += "from " + output_schema + "." + table_unique_prefix + "naics_2010 " 
#SPECIFYING THE BASE TABLE
sql_string3 += " group by name_type "
#GROUPING THE SUM for both ID

# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r



### STEP-2; DATA-B

Getting a list of all employees and their wage earnings for first quarter in 2010 from "ides.il_wage"

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

In [ ]:
#STEP 2- DATA B
#DATA B: Getting a list of all employees and their wage earnings for first quarter in 2010 from "ides.il_wage"

# generate SQL
sql_string = "CREATE table if not exists " + output_schema + "." + table_unique_prefix + "employee_2010 as"
sql_string += " SELECT DISTINCT ssn, empr_no, seinunit, ein, wage, hours, weeks"
sql_string += " from ides.il_wage "
sql_string += " where year=2010 and quarter=1"
sql_string += ";"


# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE employee_2010 created on " + str( datetime.datetime.now() ) )

#pgsql_cursor.execute("drop table data_prep.naics_2010")
# pgsql_connection.commit()

#pgsql_connection.rollback()

In [ ]:
#looking at our data
#generating read SQL
sql_string2 = "select * from " + output_schema + "." + table_unique_prefix + "employee_2010"
#sql_string2 += " limit 10"

# read it
r = pd.read_sql(sql_string2, con=pgsql_connection)
r.head(5)
#You can test the time taken to run this query by using the limit option vs not using. 
#While the limit option will lead to the query reading only 10 rows, 
#not using it will lead the query reading all of the rows- yet, reporting only 5 

#It is always better to use the limit option if we know we are not interested in reading the whole data

In [ ]:
#looking at #employees
#generating read SQL
sql_string3 = "select count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "employee_2010"
# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


In [ ]:
sql_string3 = "select count(*) from " + output_schema + "." + table_unique_prefix + "employee_2010"
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


In [ ]:
#looking at employees with non-missing wages

sql_string4 = "select count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "employee_2010"+ " where wage>0"
r = pd.read_sql(sql_string4, con=pgsql_connection)
r


### STEP-2; DATA-C

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__SQL QUERY - taking a random sample__

We want to take a random subset of 10,000 employees to shorten the running time of this query. 


In [ ]:
#Taking a random sample of employees


sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_2010_random as"
sql_string += " select * from " + output_schema + "." + table_unique_prefix + "employee_2010"
sql_string += " ORDER BY RANDOM()  "
sql_string += " LIMIT 10000"


#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_2010_random created on " + str( datetime.datetime.now() ) )
#pgsql_connection.rollback()


In [ ]:
#TESTING IF THIS IS INDEED 1000 rows


sql_string = "select count(*) from " + output_schema + "." + table_unique_prefix + "emp_2010_random"

#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

r = pd.read_sql(sql_string, con=pgsql_connection)
r
#pgsql_connection.rollback()


In [ ]:
#pgsql_connection.rollback()


### STEP-2; DATA-D

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__SQL QUERY -  Matching the Data__
- LEFT JOIN

In [ ]:
#STEP 2- DATA D
#DATA D: MATCHING DATA A & DATA B--- to get a list of employees who are working in manufacturing industry

sql_string = "CREATE TABLE if not exists " + output_schema + "." +  table_unique_prefix + "emp_manuf as"
sql_string += " select distinct a.name_legal, a.auxiliary_naics, b.* from " + output_schema + "." + table_unique_prefix + "naics_2010 as a"
sql_string += " LEFT JOIN " + output_schema + "." + table_unique_prefix + "emp_2010_random as b"  
sql_string += " on a.empr_no=b.empr_no "
sql_string += " and a.seinunit=b.seinunit "
sql_string += " and a.ein=b.ein"

#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf created on " + str( datetime.datetime.now() ) )


In [ ]:
# sql_string = "select * from " + output_schema + "." + table_unique_prefix + "EMP_MANUF"+ " LIMIT 1"
# r = pd.read_sql(sql_string, con=pgsql_connection)
# r

sql_string = "select count(*), count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "EMP_MANUF"
r = pd.read_sql(sql_string, con=pgsql_connection)
r




### STEP-2; DATA-E-1

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__SQL QUERY- __
- CASE WHEN


In [ ]:
#STEP 2- DATA E-1
#DATA E-1: Preparing the HH_MEMBER data

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem0 as"
sql_string += " select distinct ssn_hash, rootrace, ssnind ,sex"
sql_string += " from idhs.hh_member "  

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE hh_mem0 created on " + str( datetime.datetime.now() ) )


In [ ]:
# Counting number of distinct SSN vs number of rows in this data

sql_string = "select count(*), count(distinct ssn_hash) from " + output_schema + "." + table_unique_prefix + "hh_mem0"
r = pd.read_sql(sql_string, con=pgsql_connection)
r



In [ ]:
#pgsql_connection.rollback()

In [ ]:



sql_string  = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem1 as"
sql_string += " select ssn_hash,  count(*) as cnt "
sql_string += "from " + output_schema + "." + table_unique_prefix + "hh_mem0 group by ssn_hash"
sql_string += " order by cnt desc "

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()


In [ ]:
#reading the newly created table
sql_string="select * from " + output_schema + "."  + ind + "hh_mem1 limit 5"
s = pd.read_sql(sql_string, con=pgsql_connection)
s

In [ ]:
#Creating a sub table which has unique rows for each ssn
sql_string  = "Select cnt, count(*) as cnt,"
sql_string += " 100*(count(*)/(sum(count(*)) over())) as per" 
sql_string += " from "+ output_schema + "." + table_unique_prefix + "hh_mem1"
sql_string += " group by cnt"
s = pd.read_sql(sql_string, con=pgsql_connection)
s



In [ ]:
#STEP 2- DATA E-1
#DATA E-1: Creating the subsetted data table for hh_member which is cleaned to maintain one unique row per individual

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem2 as"
sql_string += " select distinct ssn_hash, rootrace, ssnind ,sex"
sql_string += " from " + output_schema + "." + table_unique_prefix + "hh_mem0 "  
sql_string += " where ssn_hash in "
sql_string += " (select ssn_hash from " + output_schema + "." + table_unique_prefix + "hh_mem1"
sql_string += " where cnt=1) "

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE hh_mem2 created on " + str( datetime.datetime.now() ) )


In [ ]:
#pgsql_connection.rollback()

### STEP-2; DATA-E-2

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__SQL QUERY-  Matching the Data__
- INNER JOIN

In [ ]:
#STEP 2- DATA E-2
#DATA E: MATCHING DATA WITH HH_MEMBERS
#WE only want information for head of households

#You can use the base data hh_mem0 as well which has not been cleaned. 
#The only difference is that then you will not be able to use the summary measure 'avg' as explained further
#keep reading!

match_table= "hh_mem2"
#match_table= "hh_mem0"


sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_manuf" + "_" +match_table+ " as"
sql_string += " select a.*, b.ssn_hash, b.rootrace, b.ssnind , b.sex"
sql_string += " from " + output_schema + "." + table_unique_prefix + "emp_manuf as a"  
sql_string += " inner join "  + output_schema + "." + table_unique_prefix + match_table  +  " as b"  
sql_string += " on a.ssn=b.ssn_hash"

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf_hh created on " + str( datetime.datetime.now() ) )


In [ ]:
#number of employees
sql_string = "select count(*) as cnt_rows, count(distinct ssn) as cnt_ind from " + output_schema + "." + table_unique_prefix + "emp_manuf" + "_" +match_table
r = pd.read_sql(sql_string, con=pgsql_connection)
r

#If you match the above data with hh_mem0 (not cleaned), you will get ## records for ## employees. 
#If you match the above data with hh_mem2 (cleaned), you will get ## records for ##-X employees. 

# sql_string = "select * from " + output_schema + "." + ind + "EMP_MANUF_HH LIMIT 2"
# s = pd.read_sql(sql_string, con=pgsql_connection)
# s



### STEP-2; DATA-F

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__Getting aggregate measures by SEX__

In [ ]:
#STEP 2- DATA F
#DATA F: Aggregating data - by sex
#WE only want information for head of households


match_table= "hh_mem2"
#match_table= "hh_mem0"

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_manuf_hh_Sex as"
sql_string += " select sex, count(distinct ssn) as cnt, sum(wage) as sum, "
sql_string += "100*(count(distinct ssn)/sum(count(distinct ssn)) over()) as per_cnt, "
sql_string += "100*(sum(wage)/ (sum(sum(wage)) over())) as per_wage"

sql_string += " from " + output_schema + "." + table_unique_prefix + "emp_manuf_" + match_table + " group by sex;"


#group by ssn, rootrace, sex
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf_hh created on" + str( datetime.datetime.now() ) )


In [ ]:
#Reading the data

#sql_string = "DROP TABLE " + output_schema + "." + ind + "EMP_MANUF_HH_Sex "

sql_string = "select * from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH_Sex "
s = pd.read_sql(sql_string, con=pgsql_connection)
s



In [ ]:

#Calculating average wages for each gender group

sql_string = "select sex, sum/cnt as avg_wage  from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH_Sex  "
s = pd.read_sql(sql_string, con=pgsql_connection)
s



### Using summary queries in SQL

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

__When to use these instead:__

Instead of actually calculating the average, we can rely on some simple SQL queries to compute mean measures. 

- Since we cleaned our data to have one row per individual- we can also rely on a simple SQL query 'average' to calculate average wages by any variable of interest. 
- However, if we had not cleaned our data-- then it will have to be necessary to actually 'calculate'- since SQL will take the number of rows- and not the number of distinct individuals (ssn) to calculate averages. 


In [ ]:
#ALTERNATIVE and QUICKER WAY TO DO THIS

sql_string = "select sex, count(distinct ssn) as ind_cnt, "
sql_string += "avg(wage)  from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH group by sex "
s = pd.read_sql(sql_string, con=pgsql_connection)
s

In [ ]:
#OTHER SUMMARY MEASURES

sql_string = "select rootrace, count(distinct ssn) as ind_cnt, avg(wage) avg from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH group by rootrace order by avg "
s = pd.read_sql(sql_string, con=pgsql_connection)
s

#  1=White, not of Hispanic origin, 
#  2=Black, not of Hispanic origin, 
#  3=American Indian or Alaskan Native, 
#  6=Hispanic (includes Mexican, Puerto Rican, Cuban, and Other South American), 
#  7=Asian or Pacific Islander (includes Indo-Chinese)

# Getting to know the IDHS database

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)

In [ ]:
#Looking at the variables of interest

# generate SQL
sql_string = " SELECT * "
sql_string += " FROM idhs.hh_member"
sql_string += " LIMIT 10"
sql_string += ";"

# read it
r = pd.read_sql(sql_string, con=pgsql_connection)
r.head(1)

## IDHS Database
## HH_Member table Descrition

If you go to the ADRF explorer, and read documentation, you will see what the different variables mean. 
Here, we paste the description of the variables we will be using for this notebook. 

1. sex- 1 for male, 2 for female
2. rootrace        
    - 1=White, not of Hispanic origin, 
    - 2=Black, not of Hispanic origin, 
    - 3=American Indian or Alaskan Native, 
    - 6=Hispanic (includes Mexican, Puerto Rican, Cuban, and Other South American), 
    - 7=Asian or Pacific Islander (includes Indo-Chinese)
3. ssn_hash- hashed SSN
4. ssnind- Indicates the validity of the recipient social security number. 
    - J=Validity unknown, 
    - K=SSA not in SSA file, 
    - L=Sex code does not match SSA file,
    - M=DOB does not match SSA file, 
    - N=DOB and sex code do not match SSA file, 
    - O=Name does not match SSA file (sex code and DOB not checked), 
    - P=SS5 form is pending with SSA, R=SS5 form is pending with enumeration control, 
    - V=SS Administration has verified SSN

In [ ]:
# sql_string1 = "SELECT sex, count( *) as cnt, count(*)/sum(count(*)) over() as PER from idhs.hh_member group by sex"
# r1 = pd.read_sql(sql_string1, con=pgsql_connection)
# print(r1)

sql_string2 = "SELECT ssnind, count( *) as cnt, 100*(count(*)/sum(count(*)) over()) as PER from idhs.hh_member group by ssnind"
r2 = pd.read_sql(sql_string2, con=pgsql_connection)
print(r2)

### Printing by INDEX

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
    
We can also print the results of our SQL queries by Column INDEX. 



In [ ]:
# Printing by INDEX

# generate SQL
sql_string = " SELECT *"
sql_string += " FROM idhs.hh_member"
sql_string += " LIMIT 10"
sql_string += ";"

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()
results=pgsql_cursor.fetchall()

#PRINTING BY INDEX

print "\nShow me the databases:\n"
for row in results: 
    print row[1]
   #print "", row[0]

### Sample Queries

- Count of Male & Females within head of household data
- Count of Individuals with no wages in a quarter
- The maximum wage in a particular industry in a quarter
   

In [ ]:
#Count of Male & Females within head of household data

# generate SQL
sql_string = " SELECT sex, count(distinct ssn_hash) as cnt "
sql_string += " FROM idhs.hh_member"
sql_string += " group by sex"
sql_string += ";"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
#Count of Individuals with no wages in a quarter

# **LONG QUERY- DO NOT RUN WHILE IN CLASS**
year='2010'
quarter='2'


# generate SQL
sql_string = " SELECT count(distinct ssn) as cnt "
sql_string += " FROM ides.il_wage"
sql_string += " where year=" + year +"and quarter= " + quarter
sql_string += " and wage=0"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
#The maximum wage and min wages in a particular time period


# **LONG QUERY- DO NOT RUN WHILE IN CLASS**
year='2010'
quarter='2'



# generate SQL
sql_string = " SELECT min(wage) as min, max(wage) as max "
sql_string += " FROM ides.il_wage"
sql_string += " where year=" + year +"and quarter= " + quarter
#sql_string += " and wage>100"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
# Close Connection and cursor
pgsql_cursor.close()
pgsql_connection.close()

print( "psycopg2 cursor and connection closed at " + str( datetime.datetime.now() ) )